# One-Pixel Attacks


---


Review of J. Su et al. paper on one pixel attacks for fooling deep neural networks

Paper: https://arxiv.org/abs/1710.08864


---



**Aims & Objectives:**


1.   Reproduce the code and findings found from this paper
2.   Build a differential evolution model to manipulate some of the best deep neural networks
3.   Use one pixel pertubations to fool networks such as VGG on the CIFAR-10 and ImageNet datasets








# Imports and Upload Image Datasets

In [16]:
!pip install py7zr gdown

In [17]:
import gdown
from google.colab import drive
import os
import py7zr
import PIL.Image
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import asarray
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import random
from tensorflow.keras.callbacks import EarlyStopping
from scipy.optimize import differential_evolution
from google.colab import drive
from tensorflow.keras.models import load_model
import math


# Downlaod and Process Input Data

In [18]:

def download(file_id, file_name, unzip=False):
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, file_name, quiet=False)
    if unzip:
        extract_dir = os.path.splitext(file_name)[0]
        os.makedirs(extract_dir, exist_ok=True)
        with py7zr.SevenZipFile(file_name, mode='r') as archive:
            archive.extractall(path=extract_dir)
        print(f"✅ Extracted '{file_name}' into folder '{extract_dir}/'")
def downloadData():
  # the IDs of the files stored on google drive
  labels = "1DieuXWsbM4x_hVUOAY7Z2GhcR1Mk0dgN"
  train = "1wOChHhwX6ANgLv2YCJpiYEtkPvFchy4G"
  #test = "12qMQYpS_4bKqroVUE-J5GCDRk41s2Tv5"
  download(labels, "trainLabels.csv")
  download(train, "train.7z", unzip=True)


def convertYVals(yvals, noOutputs=10):
   out = []
   for val in yvals:
      #arr = np.zeros(1)
      #arr[0] = val
      out.append(val)
      #out.append(np.array(val))
   out = np.array(out)
   return out


def splitData(ratio, train_data, train_labels, num_test_images, seed=42):
  yVals = convertYVals(train_labels)
  random.seed(42)

  temp = list(zip(train_data, train_labels))
  random.shuffle(temp)
  res1, res2 = zip(*temp)
  train_data, train_labels = list(res1), list(res2)

  split_index = int((ratio * len(train_data)) // 1)
  x_val, y_val = np.array(train_data[:split_index]), np.array(train_labels[:split_index])
  x_train, y_train = np.array(train_data[split_index:]), np.array(train_labels[split_index:])

  n = len(x_val) - num_test_images
  x_test, y_test = x_val[n:], y_val[n:]
  x_val, y_val = x_val[:n], y_val[:n]


  return (x_train,x_val,x_test,y_train,y_val,y_test)



def organiseData():
    downloadData()
    noImgs = len([name for name in os.listdir('train/train')])
    train_labels = np.zeros(noImgs)
    inv_keys = {"dog":0,"frog":1,"truck":2,"deer":3 , "automobile":4, "bird":5, "horse":6,"ship":7,"cat":8,"airplane":9}


    with open("trainLabels.csv", "r") as f:
        next(f)
        for line in f:
            image_id = line.split(",")[0]
            image_label = line.split(",")[1]
            train_labels[(int)(image_id)-1] = inv_keys[image_label.strip()]

    train_images=[]
    for i in range(noImgs):
        path = "train/train/"+ str(i+1) + ".png" #png 1 indexed in folder so add 1 to i
        img = PIL.Image.open(path) # Load image
        a= np.asarray(img)
        train_images.append(a)


    ratio = 0.2
    num_test_images = 500
    train_images = np.array(train_images)
    (x_train,x_val,x_test,y_train,y_val,y_test) = splitData(ratio, train_images , train_labels, num_test_images, seed=42)
    y_train, y_val, y_test = y_train.flatten(), y_val.flatten(), y_test.flatten()

    return {"x_train":x_train, "x_val":x_val,"x_test":x_test,"y_train":y_train,"y_val":y_val,"y_test":y_test}




#Load Models for Differential Algorithm

In [19]:

label_keys = {0:"dog",1:"frog", 2:"truck", 3:"deer", 4:"automobile", 5:"bird", 6:"horse", 7:"ship", 8:"cat",9:"airplane" }
inv_keys = {"dog":0,"frog":1,"truck":2,"deer":3 , "automobile":4, "bird":5, "horse":6,"ship":7,"cat":8,"airplane":9}



In [21]:
#all convolutional network
def createAllConvNetwork():
  model = keras.Sequential([
    layers.Conv2D(96, (3,3), activation='relu',padding='same',  input_shape=(32,32,3)),
    layers.BatchNormalization(),
    layers.Conv2D(96, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(96, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.Conv2D(192, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(192, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.Dropout(0.3),

    layers.Conv2D(192, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(192, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(192, (1,1), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(10, (1,1), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.AveragePooling2D(pool_size=(6, 6)),

    layers.Flatten(),
    layers.Dense(10, activation='softmax')
  ])

  # Compile the model
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

def createNetworkInNetwork():
  model = keras.Sequential([
    layers.Conv2D(192, (5,5), activation='relu',padding='same', input_shape=(32,32,3)),
    layers.BatchNormalization(),
    layers.Conv2D(160, (1,1), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(96, (1,1), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.AveragePooling2D(pool_size=(3, 3), strides=(2,2),padding='same'),
    layers.Dropout(0.5),

    layers.Conv2D(192, (5,5), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(192, (5,5), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(192, (5,5), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.AveragePooling2D(pool_size=(3, 3), strides=(2,2),padding='same'),
    layers.Dropout(0.5),

    layers.Conv2D(192, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(192, (1,1), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(10, (1,1), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.AveragePooling2D(pool_size=(8, 8),padding='same'),

    layers.Flatten(),
    layers.Dense(10, activation='softmax')
  ])


  # Compile the model
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model



#vgg network
def createVGG():
  model = keras.Sequential([
    layers.Conv2D(64, (3,3), activation='relu',padding='same',  input_shape=(32,32,3)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'),

    layers.Conv2D(128, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'),

    layers.Conv2D(256, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'),

    layers.Conv2D(512, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'),

    layers.Conv2D(512, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3,3), activation='relu',padding='same'),
    layers.BatchNormalization(),

    layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'),

    layers.Flatten(),
    layers.Dense(2048, activation='relu'),
    layers.Dense(2048, activation='relu'),
    layers.Dense(10, activation='softmax')
  ])

  # Compile the model
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model

In [27]:

def trainModels(models,data):
    EPOCHS=2
    BATCHSIZE=64
    print(models)
    history = models["nin"].fit(
        data["x_train"],
        data["y_train"],
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        validation_data=(data["x_test"], data["y_test"]),
        callbacks=[EarlyStopping(monitor='val_accuracy', patience=10, min_delta=0.001, restore_best_weights=True)]
    )
    history = models["vgg"].fit(
        data["x_train"],
        data["y_train"],
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        validation_data=(data["x_test"], data["y_test"]),
        callbacks=[EarlyStopping(monitor='val_accuracy', patience=10, min_delta=0.001, restore_best_weights=True)]
    )
    history = models["conv"].fit(
        data["x_train"],
        data["y_train"],
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        validation_data=(data["x_test"], data["y_test"]),
        callbacks=[EarlyStopping(monitor='val_accuracy', patience=10, min_delta=0.001, restore_best_weights=True)]
    )
    return models


In [34]:
def createModels(data , train=False):
    if not train:
      allconv = "1AzSxkN6oYEbwlGEyLNSFGHsMKZRkBqzh"
      nin = "1OreWkCmQFr62Ekk6wvXtSeqXIy5154es"
      vgg = "1x0clBBspx8DAQ3b1KbSROYKgaCqvO_mX"

      file_name = "nin.keras"
      url = f"https://drive.google.com/uc?id={nin}"
      gdown.download(url, file_name, quiet=False)
      ninModel = load_model(file_name)

      file_name = "allconv.keras"
      url = f"https://drive.google.com/uc?id={allconv}"
      gdown.download(url, file_name, quiet=False)
      convModel = load_model(file_name)

      file_name = "vgg.keras"
      url = f"https://drive.google.com/uc?id={vgg}"
      gdown.download(url, file_name, quiet=False)
      vggModel = load_model(file_name)

      return {"conv":convModel,"nin":ninModel, "vgg":vggModel}
    else:
      models = {"conv": createAllConvNetwork(), "nin":createNetworkInNetwork() , "vgg": createVGG()}
      models = trainModels(models, data)
      return models


In [35]:
#data=organiseData()
models=createModels(data, True)

Epoch 1/2
  3/625 ━━━━━━━━━━━━━━━━━━━━ 59:34 6s/step - accuracy: 0.1363 - loss: 2.3328  

KeyboardInterrupt: 

# Differential Evolution

Implementing differential evolution

In [9]:
def sci_pi_vectorized_one_pixel_de(
    image,
    model,
    target_class,
    bounds,
    popsize=400,
    maxiter=100,
    batch_size=400,
    crossover=0.7,
    preprocess_fn=lambda x: x.astype(np.float32),
    target=False,
    verbose=False,
  ):
    # Change the target class based on whether this is a targeted attack or not
    targeted_attack = target


    # Define bounds for a flat vector of x,y,r,g,b values
    # For more pixels, repeat this layout
    bounds = [(0,32), (0,32), (0,256), (0,256), (0,256)]

    # Population multiplier, in terms of the size of the perturbation vector x
    popmul = max(1, popsize // len(bounds))

    # Format the predict/callback functions for the differential evolution algorithm

    def predict_fn(candidates):
        # pop_candidates: (N, dims)
        N = len(candidates[0])
        copies = np.repeat(image[np.newaxis, ...], N, axis=0).astype(np.float32)
        for i in range(N):
            x = int(np.clip(round(candidates[0,i]), 0, image.shape[1]-1))
            y = int(np.clip(round(candidates[1,i]), 0, image.shape[0]-1))
            r, g, b = candidates[2,i], candidates[3,i], candidates[4,i]
            copies[i, y, x] = [r, g, b]
        # preprocess and predict in batches
        preds = []
        for start in range(0, N, batch_size):
            end = start + batch_size
            batch = preprocess_fn(copies[start:end])
            p = model.predict(batch, verbose=0)
            preds.append(p)
        preds = np.vstack(preds)  # shape (N, num_classes)
        # fitness: log prob of true_class (we minimize this)
        if target:
            for i in range(len(preds)):
              preds[i][target_class] = 1 - preds[i][target_class]
        fitness=np.log(preds[:,target_class] + 1e-12)

        return fitness

    STRATEGY="rand1bin"
    # Call Scipy's Implementation of Differential Evolution
    attack_result = differential_evolution(
        predict_fn,
        bounds, maxiter=maxiter, popsize=popmul,vectorized=True, disp=verbose,strategy=STRATEGY, polish=True, recombination=crossover,
        updating="deferred",mutation=0.5,
        #callback = callback_fn
        )
    return attack_result



In [10]:


def runAttack(targeted, target,model,img,  verbose=True):
  label_keys = {0:"dog",1:"frog", 2:"truck", 3:"deer", 4:"automobile", 5:"bird", 6:"horse", 7:"ship", 8:"cat",9:"airplane" }

  bounds = [(0,31),(0,31),(0,255),(0,255),(0,255)]
  res = sci_pi_vectorized_one_pixel_de(
      image=img,
      model = model,
      target_class=target,
      target=targeted,
      bounds=bounds,
      popsize=400,
      maxiter=100,
      preprocess_fn=lambda x: x.astype(np.float32),
  )
  # Apply one-pixel attack using result from differential evolution
  return res

In [13]:
def successfulAttack(img, res,target_class, targeted, index=-1, model_name="vgg"):

    with open("targeted.txt", "w") as f:
      f.write(f"Image Number,Predicted Class,True Class,Success,Confidence of Predicted,Confidence of Original Image Target Class,Model\n")

    # Apply one-pixel attack using result from differential evolution
    x,y= int(res.x[0]), int(res.x[1])
    r, g, b = res.x[2], res.x[3], res.x[4]
    adv_image = img.copy()
    adv_image[y, x] = [r, g, b]
    adv_pred = model.predict(adv_image[np.newaxis, ...])[0]
    adv_class = np.argmax(adv_pred)

    true_pred = model.predict(img[np.newaxis, ...])[0]
    true_class_pred = true_pred[target_class]
    adv_true_class = adv_pred[target_class]

    if(targeted):
        with open("targeted.txt", "a") as f:
            string = f"{index},{adv_class},{target_class},{adv_class == target_class},{adv_pred[adv_class]},{true_class_pred},{model_name}\n"
            print("OUTPUT:")
            print(string)
            f.write(string)

        return (adv_class ==   target_class , adv_pred, adv_true_class)
    else:

        with open("nontargeted.txt", "a") as f:
            f.write(f"{index},{adv_class},{target_class},{adv_class != target_class},{adv_pred[adv_class]},{adv_true_class},{model_name}\n")
        return (adv_class != target_class, adv_class, adv_true_class)

def nonTargeted(model, data, num, model_name="vgg"):
  with open("nontargeted.txt", "w") as f:
    f.write(f"Image Number,Predicted Class,True Class,Success,Confidence of Predicted,Confidence of True,Model\n")

  out=[]
  succCount=0
  avgClass=0
  avgTrueClass=0
  count=0
  for index in range(num):
    print(count, " Iteration")
    count+=1
    print(index)
    res = runAttack(target=int(data["y_test"][index]), targeted=False, model=model, img=data["x_test"][index], verbose=False)
    resData=successfulAttack(data["x_test"][index],res, int(data["y_test"][index]), False, index, model_name)
    if resData[0] :
      succCount+=1
    avgClass+=resData[1]
    avgTrueClass+=resData[2]
    print(resData)
    print("Success:", resData[0])
    print("Predicted Class:", resData[1])
    print("Confidence of True Class:", resData[2])
    out.append(resData)

  print("Success rate:", succCount/num)
  print("Class Confidence of highest confidence label:", avgClass/num)
  print("Average Confidence of correct label:", avgTrueClass/num)
  return out

def targeted(model, data, num, model_name="vgg", offset=0):
  out=[]
  succCount=0
  avgClass=0
  avgTrueClass=0
  count=0
  targeted=True
  for j in range(num):
    index = j + offset
    print("IMAGE NO:", str(index))
    for i in range(10):
      if(i == int(data["y_test"][index])):
        continue
      res = runAttack(target=i, targeted=targeted, model=model,img=data["x_test"][index], verbose=True)
      resData=successfulAttack(data["x_test"][index],res, i, True, index, model_name)
      print(i)
      if resData[0] :
        succCount+=1
      avgClass+=resData[1]
      avgTrueClass+=resData[2]
      print(resData)
      out.append(resData)

  print("Success rate:", succCount/num)
  print("Class Confidence of highest confidence label:", avgClass/num)
  print("Average Confidence of correct label:", avgTrueClass/num)
  return out



#Run Simulation of targeted and non targeted attck of vgg Model

In [15]:

#Run Simulation
model = models["vgg"]
nonTargeted(model, data, 100, "vgg")
targeted(model,data,20, "vgg")

IMAGE NO: 0


KeyboardInterrupt: 